# Implementation 2

The same use case as in [m7_1_Implementation_regex](m7_1_Implementation_regex.ipynb) can be implemented using pyConText.

In order to benefit from the functionality provided by the framwork, you need to ensure that the modules that you use can operate using the classes provided by the framework. For that purpose, [pyContText](https://pypi.python.org/pypi/pyConTextNLP/0.6.2.0) functionality has to be encapsulated into a Wrapper class [wrapperPyConText](wrapperPyConText_doc.html)



In [1]:
from pipeUtils import Document
from pipeUtils import Annotation

from wrapperPyConText import ConTextPipe
from PyRuSH.RuSH import RuSH

Since we are using our framework, the first lines of this implementation are identical to the regex implementation.

In [2]:
doc = Document()

doc.load_document_from_file('data/test.txt')
doc.load_annotations_from_brat('data/test.ann')

As discussed in the previous modules, pyConText operates at a sentence level, so prior to sending documents to pyConText, we need to break the text into sentences. 

[pyRuSH](https://pypi.python.org/pypi/PyRuSH) is a sentence segmentation package that works well on clinical data. It applying pyRuSH segmenter creates a list of spans for each sentence that can be converted to a list of annotations. 

In [3]:
sentence_rules='KB/rush_rules.tsv'
sentence_segmenter = RuSH(sentence_rules)

sentences=sentence_segmenter.segToSentenceSpans(doc.text)
i=0
for sentence in sentences:
    i=i+1  #incrementing the index
    doc.annotations.append(Annotation(start_index=sentence.begin,
                                      end_index=sentence.end, 
                                      type="Sentence", 
                                      ann_id='Sent_'+str(i)))
    
print(doc.toString())

test.txt
-------
General Medical Clinic
05/28/2010 13:00


CC
Follow up depression.

Subjective

Depression
The pt indicates Citalopram is helping control her depression symptoms but she continues to feel depressed most days.  Her sleep and fatigue have improved significantly with use of Citalopram.  She denies suicidal ideation.  Her PHQ-9 score is 18 today.

Hypertension
No Light-headedness.  The pt reports compliance with use of lisinopril and metoprolol.  She has been on these two medications for several years and has never used any other antihypertensive medications in her life.  She has not been checking her BP at home.

Osteoarthritis
Knee pain is well controlled currently.  No knee pain today.

Coronary Artery Disease
No angina.  No dyspnea.


Allergies
NKDA

PMH
Depression
Hypertension
Iron deficiency anemia
Osteoarthritis
Coronary Artery Disease
Hyperlipidemia

PSurgHx
None 

FamHx
None significant

SocHx
Lifetime non-user of tobacco.
Drinks alcohol rarely.
Has 5 adult childr

Specifying context rules.

In [4]:
target_rules='''
Comments: ''
Direction: ''
Lex: pain
Regex: ''
Type: Depression_Symptom
---
Comments: ''
Direction: ''
Lex: depression
Regex: 'depres\\w+'
Type: Depression_Symptom
---
Comments: ''
Direction: ''
Lex: suicidal
Regex: 'suicidal\\s*ideation'
Type: Depression_Symptom
'''
# context rules are often lengthy, you can point 
#    context_rules to an external rule files instead
context_rules='''Comments: ''
Direction: forward
Lex: 'no'
Regex: '\\bno\\b'
Type: Negation
---
Comments: ''
Direction: forward
Lex: 'denies'
Regex: ''
Type: Negation
'''

Now we have our rules specified and our sentences segmented. Let's apply rules on each sentence and add new annotations to the list

In [5]:

context_pipe = ConTextPipe(target_rules, context_rules)
new_anns = []
for a in doc.annotations:
    if a.type == "Sentence":
        b = context_pipe.process(a, doc.text)
        doc.annotations.extend(b)


In [6]:
print(doc.toString())

test.txt
-------
General Medical Clinic
05/28/2010 13:00


CC
Follow up depression.

Subjective

Depression
The pt indicates Citalopram is helping control her depression symptoms but she continues to feel depressed most days.  Her sleep and fatigue have improved significantly with use of Citalopram.  She denies suicidal ideation.  Her PHQ-9 score is 18 today.

Hypertension
No Light-headedness.  The pt reports compliance with use of lisinopril and metoprolol.  She has been on these two medications for several years and has never used any other antihypertensive medications in her life.  She has not been checking her BP at home.

Osteoarthritis
Knee pain is well controlled currently.  No knee pain today.

Coronary Artery Disease
No angina.  No dyspnea.


Allergies
NKDA

PMH
Depression
Hypertension
Iron deficiency anemia
Osteoarthritis
Coronary Artery Disease
Hyperlipidemia

PSurgHx
None 

FamHx
None significant

SocHx
Lifetime non-user of tobacco.
Drinks alcohol rarely.
Has 5 adult childr

## End of pyContext implementation